In [2]:
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
import datetime as dt  
import pandas as pd


browser = Browser('chrome', headless=True)

Base = declarative_base()

todaysDate = str(dt.date.today())


connection_string = "postgres:Yatish28$@localhost:5432/planetOnFire_db"
engine = create_engine(f'postgresql+psycopg2://{connection_string}')
Base.metadata.create_all(engine)
engine.table_names()

date_df = pd.read_sql_query('select acq_date from cleaned_df', con=engine)
date_df


# URL of page to be scraped
url = "https://earthdata.nasa.gov/earth-observation-data/near-real-time/firms/active-fire-data#ed-firms-archive"


    
# Retrieve page with the requests module
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
#print(soup.prettify())

#grab third table (TXT data)
table = soup.find_all('table')[2]
#print(table)

#grab link for 7 days data in the second column (with MODIS 1 km data)
link = table.find_all('a', text='7d')[0]['href']
print(link)       
#print(str(date_df)==todaysDate)

https://firms.modaps.eosdis.nasa.gov/active_fire/c6/text/MODIS_C6_Global_7d.csv


In [3]:
df = pd.read_csv(link)

df.tail()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,confidence,version,bright_t31,frp,daynight
105108,-31.830,147.700,317.3,1.7,1.3,2020-02-11,2355,T,64,6.0NRT,294.9,17.2,D
105109,-35.730,143.783,317.6,2.4,1.5,2020-02-11,2355,T,54,6.0NRT,297.6,19.1,D
105110,-26.718,150.555,328.5,1.4,1.2,2020-02-11,2350,T,81,6.0NRT,296.7,35.2,D
105111,-26.097,146.786,325.5,2.6,1.6,2020-02-11,2350,T,73,6.0NRT,302.5,41.7,D
105112,-29.569,149.867,316.0,1.4,1.2,2020-02-11,2350,T,60,6.0NRT,295.3,10.8,D


In [4]:
#new_df = df.loc[df["acq_date"]==todaysDate]
#new_df.tail(1)

In [5]:
cleaned_df = df.drop(columns=["scan","track","acq_time","satellite","confidence","version","frp","daynight"])
cleaned_df.tail()

,latitude,longitude,brightness,acq_date,bright_t31
105108,-31.830,147.700,317.3,2020-02-11,294.9
105109,-35.730,143.783,317.6,2020-02-11,297.6
105110,-26.718,150.555,328.5,2020-02-11,296.7
105111,-26.097,146.786,325.5,2020-02-11,302.5
105112,-29.569,149.867,316.0,2020-02-11,295.3


In [6]:
#from sqlalchemy import create_engine

connection_string = "postgres:Yatish28$@localhost:5432/planetOnFire_db"
engine = create_engine(f'postgresql+psycopg2://{connection_string}')

In [7]:
engine.table_names()

['Trial_table', 'cleaned_df', 'new_df']

In [8]:
from sqlalchemy import create_engine

# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float

In [9]:
Base.metadata.create_all(engine)
print(cleaned_df.loc[cleaned_df.index[-1], "acq_date"])

2020-02-11


In [10]:
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [11]:
#The code below is commented because when we run 
#new_df.to_sql('new_df', con=engine, if_exists='replace', index=False)

#if (cleaned_df.loc[cleaned_df.index[-1], "acq_date"]!=todaysDate):
cleaned_df.to_sql('cleaned_df', con=engine, if_exists='replace', index=False) 

In [12]:
engine.table_names()

['Trial_table', 'cleaned_df', 'new_df']

In [13]:
#scrapedData = pd.read_sql_query('select * from cleaned_df', con=engine).tail()
#scrapedData
#engine.execute("SELECT * FROM users").fetchall()

In [14]:
#longi = pd.read_sql_query('select longitude from cleaned_df', con=engine)
#dictlongi = longi.to_dict('longi')
#dictlongi


In [15]:
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
import datetime as dt  
import pandas as pd


browser = Browser('chrome', headless=True)

Base = declarative_base()

todaysDate = str(dt.date.today())


connection_string = "postgres:Yatish28$@localhost:5432/planetOnFire_db"
engine = create_engine(f'postgresql+psycopg2://{connection_string}')
Base.metadata.create_all(engine)
engine.table_names()

date_df = pd.read_sql_query('select acq_date from cleaned_df', con=engine)
date_check = date_df["acq_date"].iloc[-1]

# URL of page to be scraped
url = "https://earthdata.nasa.gov/earth-observation-data/near-real-time/firms/active-fire-data#ed-firms-archive"

if ((date_check)!=todaysDate):
    # Retrieve page with the requests module
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    #print(soup.prettify())

    #grab third table (TXT data)
    table = soup.find_all('table')[2]
    #print(table)

    #grab link for 7 days data in the second column (with MODIS 1 km data)
    link = table.find_all('a', text='7d')[0]['href']
    #print(link)
    
    df = pd.read_csv(link)
    
    cleaned_df = df.drop(columns=["scan","track","acq_time","satellite","confidence","version","frp","daynight"])
    
    cleaned_df.to_sql('cleaned_df', con=engine, if_exists='replace', index=False) 
    print ("scraped")

else:
    print("table exists")
Mdata = pd.read_sql_query('select * from cleaned_df', con=engine).tail()
Mdata

table exists


,latitude,longitude,brightness,acq_date,bright_t31
105108,-31.830,147.700,317.3,2020-02-11,294.9
105109,-35.730,143.783,317.6,2020-02-11,297.6
105110,-26.718,150.555,328.5,2020-02-11,296.7
105111,-26.097,146.786,325.5,2020-02-11,302.5
105112,-29.569,149.867,316.0,2020-02-11,295.3


In [16]:
date_df = pd.read_sql_query('select acq_date from cleaned_df', con=engine)
date_check = date_df["acq_date"].iloc[-1]
date_check

'2020-02-11'

In [17]:
todaysDate

'2020-02-11'